In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install protobuf==3.20.3
!pip install mediapipe opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.14 requires protobuf<5,>=4.25.3, but you have protobuf 3.20.3 which is incompatible.


  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 409, in resolve
    unsatisfied_

In [ ]:
import cv2
import mediapipe as mp
import os
import csv

# MediaPipe 모듈 초기화
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 디렉토리 설정
video_directory = '/content/drive/MyDrive/DL/output/이상행동데이터/절도'  # 비디오 파일이 있는 디렉토리 경로
output_directory = '/content/drive/MyDrive/DL/output/이상행동데이터/절도/landmark_x'  # 랜드마크가 그려진 비디오를 저장할 디렉토리 경로
data_directory = '/content/drive/MyDrive/DL/output/이상행동데이터/절도/landmark_data_x'  # 랜드마크 데이터를 저장할 디렉토리 경로

In [ ]:

# 디렉토리 없으면 생성
os.makedirs(output_directory, exist_ok=True)
os.makedirs(data_directory, exist_ok=True)


In [ ]:
import cv2
import csv
import mediapipe as mp
import os

# Mediapipe Pose 모델 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 디렉토리 설정
video_directory = '/content/drive/MyDrive/DL/output/이상행동데이터/절도'  # 비디오 파일이 있는 디렉토리 경로
output_directory = '/content/drive/MyDrive/DL/output/이상행동데이터/절도/landmark'  # 랜드마크가 그려진 비디오를 저장할 디렉토리 경로
data_directory = '/content/drive/MyDrive/DL/output/이상행동데이터/절도/landmark_data'  # 랜드마크 데이터를 저장할 디렉토리 경로

# 비디오 파일 목록 가져오기
clip_files = os.listdir(video_directory)

for filename in clip_files:
    video_filepath = os.path.join(video_directory, filename)
    cap = cv2.VideoCapture(video_filepath)

    # 동영상 저장 설정
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_filepath = os.path.join(output_directory, filename)
    out = cv2.VideoWriter(output_filepath, fourcc, fps, (width, height))

    # CSV 파일 경로
    csv_filepath = os.path.join(data_directory, filename.replace('.mp4', '.csv'))
    with open(csv_filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['frame', 'landmark_type', 'x', 'y', 'z', 'visibility'])

        frame_idx = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pose_results = pose.process(image_rgb)

            # 포즈 랜드마크 저장 및 그리기
            if pose_results.pose_landmarks:
                for idx, landmark in enumerate(pose_results.pose_landmarks.landmark):
                    writer.writerow([frame_idx, f'pose_{idx}', landmark.x, landmark.y, landmark.z, landmark.visibility])
                mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # 결과 프레임 저장
            out.write(frame)
            frame_idx += 1

    cap.release()
    out.release()

pose.close()

print("랜드마크 데이터 및 동영상 저장 완료!")


랜드마크 데이터 및 동영상 저장 완료!
